In [54]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
mcc = MouseConnectivityCache(manifest_file='connectivity/mouse_connectivity_manifest.json')
import pandas as pd

# grab the StructureTree instance
structure_tree = mcc.get_structure_tree()

In [55]:
all_edges = pd.read_csv('./whole brain/edge list - projection volume.csv')

In [56]:
# Get list of all nodes involved in feeding from old feeding doc
feeding = pd.read_excel('../../spreadsheets/Experiment Nodes Feeding List.xlsx')
feeding_nodes = feeding.loc[feeding['Associated with feeding?'].str.contains('Y', na=False)][['structure-id', 'structure-abbrev', 'structure-name']]

### Knockdown some nodes in the network

In [101]:
# Knockdown some nodes in the network
#knockdown_nodes = [63, 250, 754, 38, 780, 647]
kdn = 'no'
knockdown_nodes = []
feeding_nodes = feeding_nodes.loc[~feeding_nodes['structure-id'].isin(knockdown_nodes)]

### Create list of all feeding edges

In [102]:
# Create new dataframe with only feeding edges
feeding_edges = all_edges.loc[(all_edges['origin_id'].isin(feeding_nodes['structure-id'])) | (all_edges['target_id'].isin(feeding_nodes['structure-id']))]
print('There are %d edges involving the feeding circuit.' % feeding_edges.shape[0])
feeding_edges.head()

There are 4539 edges involving the feeding circuit.


,experiment_id,id,origin_abbrev,origin_id,target_abbrev,target_id,is_injection,max_voxel_density,max_voxel_x,max_voxel_y,...,normalized_projection_volume,projection_density,projection_energy,projection_intensity,projection_volume,sum_pixel_intensity,sum_pixels,sum_projection_pixel_intensity,sum_projection_pixels,volume
16,158258062,628588727,VMH,693,root,997,True,1.0,6610,6100,...,1.000000,0.991973,12449.342253,12550.076539,1.465075,1.501513e+13,1.205657e+09,1.500964e+13,1.195980e+09,1.476930
19,158258062,628588946,VMH,693,HY,1097,True,1.0,6610,6100,...,0.974524,0.994024,12568.514526,12644.075595,1.427751,1.473899e+13,1.172518e+09,1.473681e+13,1.165511e+09,1.436335
20,158258062,628589030,VMH,693,IB,1129,True,1.0,6610,6100,...,0.974524,0.994024,12568.512992,12644.074785,1.427751,1.473899e+13,1.172518e+09,1.473681e+13,1.165511e+09,1.436335
21,158258062,628586285,VMH,693,BS,343,True,1.0,6610,6100,...,0.974524,0.994024,12568.512992,12644.074785,1.427751,1.473899e+13,1.172518e+09,1.473681e+13,1.165511e+09,1.436335
22,158258062,628584332,VMH,693,grey,8,True,1.0,6610,6100,...,0.974524,0.994024,12568.512992,12644.074785,1.427751,1.473899e+13,1.172518e+09,1.473681e+13,1.165511e+09,1.436335


# Top 50% projection volume analysis

### Select top 50% of edges

In [103]:
# Get top 50% of feeding edges
#feeding_edges_50 = feeding_edges.iloc[:round(feeding_edges.shape[0] / 2)]
#print('There are %d edges in the top 50%% volumes for feeding' % feeding_edges_50.shape[0])
#feeding_edges_50.head()

#### Create 3 groups of feeding network edges
Outgoing edges : Edges which start *in* the network and *leave*

Incoming edges : Edges which start *outside* the network and *enter*

Internal edges : Edges that start *in* the network and *end there*

In [104]:
# Find list of outgoing/incoming/internal edges
outgoing_edges = feeding_edges.loc[~feeding_edges['target_id'].isin(feeding_nodes['structure-id'])]  # The target is not in the feeding network
incoming_edges = feeding_edges.loc[(~feeding_edges['origin_id'].isin(feeding_nodes['structure-id'])) & (feeding_edges['target_id'].isin(feeding_nodes['structure-id']))]  # The origin isn't in the feeding network and the target is
internal_edges = feeding_edges.loc[(feeding_edges['origin_id'].isin(feeding_nodes['structure-id'])) & (feeding_edges['target_id'].isin(feeding_nodes['structure-id']))]  # Both the origin and target are in the feeding network

external_edges = pd.concat([outgoing_edges, incoming_edges])

In [105]:
internal_edges

,experiment_id,id,origin_abbrev,origin_id,target_abbrev,target_id,is_injection,max_voxel_density,max_voxel_x,max_voxel_y,...,normalized_projection_volume,projection_density,projection_energy,projection_intensity,projection_volume,sum_pixel_intensity,sum_pixels,sum_projection_pixel_intensity,sum_projection_pixels,volume
686,175265301,630010199,IRN,136,MV,202,True,1.000000e+00,11060,5260,...,2.621136e-01,0.694706,1260.189499,1813.988972,3.049572e-01,4.916998e+11,3.583452e+08,4.515829e+11,2.489447e+08,4.389729e-01
728,165972951,629617054,BMA,319,CEA,536,True,1.000000e+00,6030,6040,...,2.476374e-01,0.849592,6532.132660,7688.550322,2.919403e-01,2.105783e+12,2.805094e+08,1.832324e+12,2.383186e+08,3.436240e-01
823,180709230,630418124,ORBl,723,AIv,119,True,1.000000e+00,3170,4600,...,3.288270e-01,0.999153,14243.600631,14255.673137,2.621197e-01,3.050814e+12,2.141566e+08,3.050362e+12,2.139753e+08,2.623419e-01
874,180709230,630418091,ORBl,723,AId,104,True,1.000000e+00,3010,4370,...,3.021429e-01,0.998985,12661.320933,12674.180112,2.408489e-01,2.492421e+12,1.968110e+08,2.491887e+12,1.966113e+08,2.410935e-01
877,113935990,633239955,BMA,319,MEA,403,True,1.000000e+00,7100,6320,...,3.001956e-01,0.998499,4680.028387,4687.062077,2.387995e-01,9.139711e+11,1.952313e+08,9.136882e+11,1.949384e+08,2.391584e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13405,100141597,631382340,MS,564,BST,351,True,4.174973e-04,4880,5190,...,1.201903e-09,0.885626,1242.987395,1403.512110,4.727649e-10,5.651700e+02,4.357712e-01,5.416581e+02,3.859305e-01,5.338198e-10
13417,574363940,635385543,PSV,7,IRN,136,True,3.250572e-04,10580,6400,...,3.051300e-07,0.005595,4.856128,868.000040,3.583755e-10,2.205826e+04,5.229160e+01,2.539347e+02,2.925515e-01,6.405721e-08
13463,204908781,630572738,PVT,149,TRS,581,True,7.466386e-07,5540,4190,...,2.121136e-11,0.482759,1101.931032,2282.571546,8.231690e-13,2.123824e+00,1.391948e-03,1.533830e+00,6.719747e-04,1.705136e-12
13481,286319739,631497971,NDB,596,AAA,23,True,0.000000e+00,0,0,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,8.714411e+04,1.273160e+02,0.000000e+00,0.000000e+00,1.559621e-07


In [106]:
# For each node, count up # of escape edges it participates in
feeding_nodes['(outgoing edges)'] = feeding_nodes.apply(lambda row: outgoing_edges.loc[outgoing_edges['origin_id'] == row['structure-id']].shape[0], axis=1)
# For each node, count up # of nonescape edges it participates in
feeding_nodes['(incoming edges)'] = feeding_nodes.apply(lambda row: incoming_edges.loc[incoming_edges['target_id'] == row['structure-id']].shape[0], axis=1)
# For each node, count up # of internal edges it participates in
feeding_nodes['internal edges'] = feeding_nodes.apply(lambda row: internal_edges.loc[(internal_edges['origin_id'] == row['structure-id']) | (internal_edges['target_id'] == row['structure-id'])].shape[0], axis=1)

# Add up edges that communicate w/ outside feeding network
feeding_nodes['external edges'] = feeding_nodes['(outgoing edges)'] + feeding_nodes['(incoming edges)']
feeding_nodes.head()

,structure-id,structure-abbrev,structure-name,(outgoing edges),(incoming edges),internal edges,external edges
3,746,ORBvl,"Orbital area, ventrolateral part",59,3,13,62
4,596,NDB,Diagonal band nucleus,61,4,19,65
6,295,BLA,Basolateral amygdalar nucleus,78,8,8,86
7,872,DR,Dorsal nucleus raphe,44,7,5,51
9,693,VMH,Ventromedial hypothalamic nucleus,108,6,15,114


In [107]:
with pd.ExcelWriter('./feeding/feeding network 100 percent volume ' + str(kdn) + ' knockdown.xlsx') as writer:
    feeding_edges[['origin_abbrev', 'origin_id', 'target_abbrev', 'target_id', 'projection_volume']].to_excel(writer, sheet_name='Feeding Network', index=False)
    internal_edges[['origin_abbrev', 'origin_id', 'target_abbrev', 'target_id', 'projection_volume']].to_excel(writer, sheet_name='Internal Edges', index=False)
    external_edges[['origin_abbrev', 'origin_id', 'target_abbrev', 'target_id', 'projection_volume']].to_excel(writer, sheet_name='External Edges', index=False)
    feeding_nodes.to_excel(writer, sheet_name='Analysis', index=False)

In [108]:
# # Get top 50% of feeding network
feeding_edges_50 = feeding_edges.iloc[:round(feeding_edges.shape[0] / 2)]
print('There are %d edges in the top 50%% volumes for feeding' % feeding_edges_50.shape[0])
feeding_edges_50.head()

There are 2270 edges in the top 50% volumes for feeding


,experiment_id,id,origin_abbrev,origin_id,target_abbrev,target_id,is_injection,max_voxel_density,max_voxel_x,max_voxel_y,...,normalized_projection_volume,projection_density,projection_energy,projection_intensity,projection_volume,sum_pixel_intensity,sum_pixels,sum_projection_pixel_intensity,sum_projection_pixels,volume
16,158258062,628588727,VMH,693,root,997,True,1.0,6610,6100,...,1.000000,0.991973,12449.342253,12550.076539,1.465075,1.501513e+13,1.205657e+09,1.500964e+13,1.195980e+09,1.476930
19,158258062,628588946,VMH,693,HY,1097,True,1.0,6610,6100,...,0.974524,0.994024,12568.514526,12644.075595,1.427751,1.473899e+13,1.172518e+09,1.473681e+13,1.165511e+09,1.436335
20,158258062,628589030,VMH,693,IB,1129,True,1.0,6610,6100,...,0.974524,0.994024,12568.512992,12644.074785,1.427751,1.473899e+13,1.172518e+09,1.473681e+13,1.165511e+09,1.436335
21,158258062,628586285,VMH,693,BS,343,True,1.0,6610,6100,...,0.974524,0.994024,12568.512992,12644.074785,1.427751,1.473899e+13,1.172518e+09,1.473681e+13,1.165511e+09,1.436335
22,158258062,628584332,VMH,693,grey,8,True,1.0,6610,6100,...,0.974524,0.994024,12568.512992,12644.074785,1.427751,1.473899e+13,1.172518e+09,1.473681e+13,1.165511e+09,1.436335


In [109]:
# Find list of outgoing/incoming/internal edges
outgoing_edges_50 = feeding_edges.loc[~feeding_edges['target_id'].isin(feeding_nodes['structure-id'])]  # The target is not in the feeding network
outgoing_edges_50 = outgoing_edges_50.iloc[:round(outgoing_edges.shape[0] / 2)]
incoming_edges_50 = feeding_edges.loc[(~feeding_edges['origin_id'].isin(feeding_nodes['structure-id'])) & (feeding_edges['target_id'].isin(feeding_nodes['structure-id']))]  # The origin isn't in the feeding network and the target is
incoming_edges_50 = incoming_edges_50.iloc[:round(incoming_edges.shape[0] / 2)]
internal_edges_50 = feeding_edges.loc[(feeding_edges['origin_id'].isin(feeding_nodes['structure-id'])) & (feeding_edges['target_id'].isin(feeding_nodes['structure-id']))]  # Both the origin and target are in the feeding network
internal_edges_50 = internal_edges_50.iloc[:round(internal_edges.shape[0] / 2)]

external_edges_50 = pd.concat([outgoing_edges_50, incoming_edges_50])

In [110]:
feeding_nodes_50 = feeding_nodes
# For each node, count up # of escape edges it participates in
feeding_nodes_50['(outgoing edges)'] = feeding_nodes_50.apply(lambda row: outgoing_edges_50.loc[outgoing_edges_50['origin_id'] == row['structure-id']].shape[0], axis=1)
# For each node, count up # of nonescape edges it participates in
feeding_nodes_50['(incoming edges)'] = feeding_nodes_50.apply(lambda row: incoming_edges_50.loc[incoming_edges_50['target_id'] == row['structure-id']].shape[0], axis=1)
# For each node, count up # of internal edges it participates in
feeding_nodes_50['internal edges'] = feeding_nodes_50.apply(lambda row: internal_edges_50.loc[(internal_edges_50['origin_id'] == row['structure-id']) | (internal_edges_50['target_id'] == row['structure-id'])].shape[0], axis=1)

# Add up edges that communicate w/ outside feeding network
feeding_nodes_50['external edges'] = feeding_nodes_50['(outgoing edges)'] + feeding_nodes_50['(incoming edges)']
feeding_nodes_50.head()

,structure-id,structure-abbrev,structure-name,(outgoing edges),(incoming edges),internal edges,external edges
3,746,ORBvl,"Orbital area, ventrolateral part",34,1,8,35
4,596,NDB,Diagonal band nucleus,31,1,11,32
6,295,BLA,Basolateral amygdalar nucleus,46,3,5,49
7,872,DR,Dorsal nucleus raphe,13,4,2,17
9,693,VMH,Ventromedial hypothalamic nucleus,47,3,10,50


In [111]:
with pd.ExcelWriter('./feeding/feeding network 50 percent volume ' + str(kdn) +' knockdown.xlsx') as writer:
    feeding_edges_50[['origin_abbrev', 'origin_id', 'target_abbrev', 'target_id', 'projection_volume']].to_excel(writer, sheet_name='Feeding Network', index=False)
    internal_edges_50[['origin_abbrev', 'origin_id', 'target_abbrev', 'target_id', 'projection_volume']].to_excel(writer, sheet_name='Internal Edges', index=False)
    external_edges_50[['origin_abbrev', 'origin_id', 'target_abbrev', 'target_id', 'projection_volume']].to_excel(writer, sheet_name='External Edges', index=False)
    feeding_nodes.to_excel(writer, sheet_name='Analysis', index=False)